In [39]:
import argparse
import json
import os   
import shutil
import subprocess
from typing import Any, Dict
import sys
import pandas as pd
from eval_utils import ie_eval
import pathlib
from pathlib import Path
import pandas as pd
import itertools
from eval_utils import relation_matching
import spacy 

root_path = Path("../coviddata")
anno_dir = root_path / "Annotations"
annotator_files = list(anno_dir.glob('*.tsv'))
print(annotator_files)

[PosixPath('../coviddata/Annotations/arezou_annotations.tsv'), PosixPath('../coviddata/Annotations/madeline_annotations.tsv'), PosixPath('../coviddata/Annotations/jeff_annotations.tsv'), PosixPath('../coviddata/Annotations/kristina_annotations.tsv'), PosixPath('../coviddata/Annotations/megan_annotations.tsv'), PosixPath('../coviddata/Annotations/megan_corrections.tsv'), PosixPath('../coviddata/Annotations/madeline_corrections.tsv')]


In [40]:
golddf = pd.read_csv(annotator_files[-1], sep="\t",header=None, names=["id","text","arg0","arg1","rel","y"])
golddf = golddf[golddf["y"]=="accept"]
golddf["rel"][golddf["rel"]=="DO"] ="MECHANISM"
golddf["rel"][golddf["rel"].str.contains("USED")] = "MECHANISM"




In [38]:
golddf

,id,text,arg0,arg1,rel,y
0,at9jg710_abstract,The N - terminal domain of the coronavirus nuc...,fold,N - terminal domain of the coronavirus nucleoc...,MECHANISM,accept
1,at9jg710_abstract,This domain was shown to interact with the gen...,domain,interact with the genomic RNA,MECHANISM,accept
2,at9jg710_abstract,This domain was shown to interact with the gen...,interact with the genomic RNA,coronavirus infectious bronchitis virus ( IBV ),MECHANISM,accept
3,at9jg710_abstract,This domain was shown to interact with the gen...,interact with the genomic RNA,severe acute respiratory syndrome coronavirus ...,MECHANISM,accept
4,at9jg710_abstract,"Based on its 3D structure , we used site - dir...",site - directed mutagenesis,identify residues,MECHANISM,accept
...,...,...,...,...,...,...
102,fu0v9fch_abstract,"Recently , chronic immune reactivity has been ...",loss of RPE immunosuppressive factors,generation of excessive retinal inflammation,EFFECT,accept
103,fu0v9fch_abstract,"Herein , we summarize the varied interactions ...",RPE cell,immune response,MECHANISM,accept
104,lviulxuk_abstract,The aims of this study were to investigate the...,human bocavirus ( HBoV ),acute respiratory infections ( ARI ),EFFECT,accept
105,lviulxuk_abstract,"Nasopharyngeal samples from 300 patients , pre...",qPCR,NSP1 and NP-1 genes,MECHANISM,accept


In [15]:
lens0gold = golddf["arg0"].str.split().str.len()
lens1gold = golddf["arg1"].str.split().str.len()

In [16]:
lens1gold.quantile(0.9)

7.0

In [9]:
for annofile in [a for a in annotator_files if "mad" not in a.name]:
    print(annofile)
    #map USED/DO to MECH
    annodf = pd.read_csv(annofile, sep="\t",header=None, names=["id","text","arg0","arg1","rel","y"])
    lens0anno= annodf["arg0"].str.split().str.len()
    lens1anno = annodf["arg1"].str.split().str.len()
    lens0gold.quantile(0.9)
    print(lens0anno.quantile(0.95))
    print(lens1anno.quantile(0.95))

../coviddata/Annotations/arezou_annotations.tsv
16.19999999999999
28.799999999999983
../coviddata/Annotations/madeline_annotations.tsv
8.349999999999909
9.0
../coviddata/Annotations/jeff_annotations.tsv
9.849999999999994
14.0
../coviddata/Annotations/kristina_annotations.tsv
17.19999999999999
25.0
../coviddata/Annotations/megan_annotations.tsv
19.0
21.0
../coviddata/Annotations/megan_corrections.tsv
7.349999999999994
8.0
../coviddata/Annotations/madeline_corrections.tsv
7.699999999999989
8.0


In [17]:
annotator_files

[PosixPath('../coviddata/Annotations/arezou_annotations.tsv'),
 PosixPath('../coviddata/Annotations/madeline_annotations.tsv'),
 PosixPath('../coviddata/Annotations/jeff_annotations.tsv'),
 PosixPath('../coviddata/Annotations/kristina_annotations.tsv'),
 PosixPath('../coviddata/Annotations/megan_annotations.tsv'),
 PosixPath('../coviddata/Annotations/megan_corrections.tsv'),
 PosixPath('../coviddata/Annotations/madeline_corrections.tsv')]

In [18]:
def get_match(pair, text, collapse,jaccard_thresh,match_metric,filter_stop,seen_pred_gold,good_preds,span_mode=False):
        if collapse:
            labels = [1,1]
        else:
            labels = [pair[0][2],pair[1][2]]
        m = relation_matching(pair,metric=match_metric, labels = labels,thresh=jaccard_thresh, filter_stop = filter_stop, span_mode= span_mode)
        if m and ((i,pair[0][0],pair[0][1],pair[1][0],pair[1][1]) not in seen_pred_gold):
            #good_preds.append([i,pair[0],pair[1]])
            good_preds.append((i,tuple(pair[0]),tuple(pair[1]),text))
            seen_pred_gold[(i,pair[0][0],pair[0][1],pair[1][0],pair[1][1])]=1

In [45]:

jaccard_thresh = 0.3
#iterate over annotator files except gold
for annofile in annotator_files:#[a for a in annotator_files if "mad" not in a.name]:
    print(annofile)
    #map USED/DO to MECH
    annodf = pd.read_csv(annofile, sep="\t",header=None, names=["id","text","arg0","arg1","rel","y"])
    annodf = annodf[annodf["y"]=="accept"]
    lens0anno= annodf["arg0"].str.split().str.len()
    lens1anno = annodf["arg1"].str.split().str.len()
    
    annodf["rel"][annodf["rel"]=="DO"] ="MECHANISM"
    annodf["rel"][annodf["rel"].str.contains("USED")] = "MECHANISM"
    overlap = set(annodf.id).intersection(set(golddf.id))
    #use only overalpping documents
    annodf = annodf[annodf["id"].isin(overlap)]
    golddf = golddf[golddf["id"].isin(overlap)]

    goldrels = golddf[["id","arg0","arg1","rel"]]#.drop_duplicates()
    goldrels = goldrels.drop_duplicates(subset =["id","arg0","arg1"]).set_index("id")
    if "conf" in annodf.columns:
        predrels = annodf[["id","arg0","arg1","rel","conf"]].drop_duplicates(subset =["id","arg0","arg1"]).set_index("id",inplace=False)
    else:
        predrels = annodf[["id","arg0","arg1","rel"]].drop_duplicates(subset =["id","arg0","arg1"]).set_index("id",inplace=False)
    
    good_preds1 = []
    seen_pred_gold1 = {}
    good_preds2 = []
    seen_pred_gold2 = {}

    match_count = 0
    for i in predrels.index.unique():
        if i in goldrels.index.unique():
            gold = goldrels.loc[i]
            text = golddf[golddf.id == i]["text"].unique()
            if type(predrels.loc[i]) == pd.core.series.Series:
                preds = [predrels.loc[i].values]
            else:
                preds = predrels.loc[i].values
            c = list(itertools.product(gold.values, preds))
            #for each pair, find gold-annotation matches according to different criteria (e.g., collapse true/false or match_metric jaccard/substring)
            #compute agreement 
            #find the diff between two criteria for each annotator and save to disk
            for pair in c:
                collapse = True
                get_match(pair,text = text,collapse=collapse,jaccard_thresh=jaccard_thresh,match_metric='substring',filter_stop=False,seen_pred_gold = seen_pred_gold1,\
                good_preds=good_preds1)
                collapse = False
                get_match(pair, text =text,collapse=collapse,jaccard_thresh= jaccard_thresh,match_metric="substring",filter_stop=False,seen_pred_gold = seen_pred_gold2,\
                good_preds = good_preds2)

    print('collapsed: {0} metric: {1} agree: {2} total correct preds: {3} total gold: {4}'.format(True, 'substring',2*len(good_preds1)/(len(goldrels) + len(predrels)),
    len(good_preds1), len(goldrels))) 
    print('collapsed: {0} metric: {1} agree: {2} total correct preds: {3} total gold: {4}'.format(True, 'substring',2*len(good_preds2)/(len(goldrels) + len(predrels)),
    len(good_preds2), len(goldrels)))
    print('***')
    diff_csv = []
    for g in good_preds1:
        if tuple(g) not in good_preds2:
            diff_csv.append([g[0],g[3],g[1][0],g[1][1],g[1][2],g[2][0],g[2][1],g[2][2]])
    
    pd.DataFrame(diff_csv,columns = ["abstract_id","text","gold_arg0","gold_arg1","gold_label","anno_arg0","anno_arg1","anno_label"]).to_csv(annofile.name+"_diff_gold.csv",header=True,index=False,sep="\t")

../coviddata/Annotations/arezou_annotations.tsv
collapsed: True metric: substring agree: 0.5373134328358209 total correct preds: 36 total gold: 66
collapsed: True metric: substring agree: 0.40298507462686567 total correct preds: 27 total gold: 66
***
../coviddata/Annotations/madeline_annotations.tsv
collapsed: True metric: substring agree: 0.8455284552845529 total correct preds: 52 total gold: 66
collapsed: True metric: substring agree: 0.8292682926829268 total correct preds: 51 total gold: 66
***
../coviddata/Annotations/jeff_annotations.tsv
collapsed: True metric: substring agree: 0.5454545454545454 total correct preds: 39 total gold: 66
collapsed: True metric: substring agree: 0.48951048951048953 total correct preds: 35 total gold: 66
***
../coviddata/Annotations/kristina_annotations.tsv
collapsed: True metric: substring agree: 0.6862745098039216 total correct preds: 35 total gold: 66
collapsed: True metric: substring agree: 0.5882352941176471 total correct preds: 30 total gold: 66


In [11]:
pair

(array(['PROBE', 'commercial eXplosives Identification Tool', 'MECHANISM'],
       dtype=object),
 array(['( XIT )',
        'transform disparate , often paper - based , information on commercial explosives and articles into a standardized electronic database',
        'MECHANISM'], dtype=object))

In [3]:

hypothesis = "Alanine substitution"

reference = "Alanine substitution of either Arg-76 or Tyr-94 in the N - terminal domain of IBV N protein"

rouge = Rouge()
scores = rouge.get_scores(reference, hypothesis)


In [5]:
scores = rouge.get_scores("r", "b")


In [6]:
scores

[{'rouge-1': {'f': 0.0, 'p': 0.0, 'r': 0.0},
  'rouge-2': {'f': 0.0, 'p': 0.0, 'r': 0.0},
  'rouge-l': {'f': 0.0, 'p': 0.0, 'r': 0.0}}]

In [9]:
doc = nlp(pair[0][1])
[t for t in doc if t.dep_ =="ROOT"]

In [13]:
doc

commercial eXplosives Identification Tool

In [12]:
[t for t in doc if t.dep_ =="ROOT"]

[Tool]

In [14]:
pair[1]

array(['( XIT )',
       'transform disparate , often paper - based , information on commercial explosives and articles into a standardized electronic database',
       'MECHANISM'], dtype=object)